In [2]:
# %pip install sklearn

In [3]:
import numpy as np
import xgboost as xgb
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt
# from sklearn.metrics import r2_score
# from sklearn.model_selection import cross_val_score, KFold
from xgboost import DMatrix


### Input Data - Cleaned Dataset All

In [4]:

# df_TagDesc = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\CSV\Not for Processing\TagDesc.csv')

# df_All_1 = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\1 Preprocess\Merge Data\contData_all_Avg_1o2.csv')
# df_All_2 = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\1 Preprocess\Merge Data\contData_all_Avg_2o2.csv')
# # Concatenate (union) the dataframes
# df_All = pd.concat([df_All_1, df_All_2], ignore_index=True)

df_rC4_1_75 = pd.read_csv(r'C:\Users\steve\OneDrive\1. BAIUTEK\Project-OptiC4\IV Optimize\Statistical Testing Sets\rC4_1_75.csv')
df_rC4_5_75 = pd.read_csv(r'C:\Users\steve\OneDrive\1. BAIUTEK\Project-OptiC4\IV Optimize\Statistical Testing Sets\rC4_5_75.csv')




In [5]:
print(df_rC4_1_75.info())
print(df_rC4_5_75.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47882 entries, 0 to 47881
Data columns (total 26 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Date                47882 non-null  object 
 1   425_pct_Al          47882 non-null  float64
 2   C4_pct_Eth          47882 non-null  float64
 3   C4_pct_H2O          47882 non-null  float64
 4   HydWtr_pct_Ammonia  47882 non-null  float64
 5   HydWtr_Na2O         47882 non-null  float64
 6   Butanol             47882 non-null  float64
 7   Decanol             47882 non-null  float64
 8   DI55102             47882 non-null  float64
 9   DI55152             47882 non-null  float64
 10  FC55003             47882 non-null  float64
 11  FC55009             47882 non-null  float64
 12  FC55552             47882 non-null  float64
 13  FC55569             47882 non-null  float64
 14  FC55576             47882 non-null  float64
 15  FFC55555            47882 non-null  float64
 16  LC55

In [6]:
print(df_rC4_1_75.head())
print(df_rC4_5_75.head())

                  Date  425_pct_Al  C4_pct_Eth  C4_pct_H2O  \
0  2012-05-16 16:00:00    6.307630    2.508420    21.96340   
1  2012-05-16 18:00:00    6.308335    2.511135    21.96865   
2  2012-05-16 19:00:00    6.309390    2.515210    21.97655   
3  2012-05-16 20:00:00    6.310090    2.517925    21.98185   
4  2012-05-16 21:00:00    6.310790    2.520640    21.98715   

   HydWtr_pct_Ammonia  HydWtr_Na2O    Butanol   Decanol   DI55102   DI55152  \
0            0.908765     2.712180  54.858300  0.722511  0.970496  0.925255   
1            0.907643     2.805415  54.858300  0.722511  0.970496  0.925255   
2            0.905959     2.945270  54.858300  0.722511  0.970496  0.925255   
3            0.904837     3.038505  51.190050  0.733904  0.970653  0.923773   
4            0.903715     3.131740  48.744567  0.741499  0.970307  0.924103   

   ...    LC55557    LC90366    LC90368   PI55004   PI55020  TC55552  \
0  ...  65.873400  61.011100  51.234400  1.306310 -0.607993  177.188   
1  ...  

# 1 75

In [7]:
# Calculate the differences and create new columns
df_rC4_1_75['FFC55555_diff'] = df_rC4_1_75['FFC55555'] - df_rC4_1_75['FFC55555_adj']
df_rC4_1_75['Butanol_diff'] = df_rC4_1_75['Butanol'] - df_rC4_1_75['Butanol_adj']
df_rC4_1_75['Decanol_diff'] = df_rC4_1_75['Decanol'] - df_rC4_1_75['Decanol_adj']
df_rC4_1_75['rC4_diff'] = (df_rC4_1_75['FFC55555']*df_rC4_1_75[''])-(df_rC4_1_75['FFC55555_adj']*df_rC4_1_75['']))

In [9]:

df_rC4_1_75['Date'] = pd.to_datetime(df_rC4_1_75['Date'])
df_rC4_1_75.set_index('Date', inplace=True)


In [16]:
# Sum up the hourly data to get daily totals for each component
daily_totals_1_75 = df_rC4_1_75[['Butanol_diff', 'Decanol_diff']].resample('D').sum()

In [17]:
# Calculate the average daily production
average_daily_totals_1_75 = daily_totals_1_75.mean()

# Count the actual number of days with data
actual_days_with_data = daily_totals_1_75.dropna().shape[0]  # dropna() ensures you count only the days with data

# Estimate annual production based on actual data days
estimated_annual_production_butanol_1_75 = average_daily_totals_1_75['Butanol_diff'] * actual_days_with_data
estimated_annual_production_decanol_1_75 = average_daily_totals_1_75['Decanol_diff'] * actual_days_with_data


print(f"Estimated Annual Production for Butanol_diff: {estimated_annual_production_butanol_1_75}")
print(f"Estimated Annual Production for Decanol_diff: {estimated_annual_production_decanol_1_75}")

Estimated Annual Production for Butanol_diff: -5747.595241433343
Estimated Annual Production for Decanol_diff: -1410.6846121160122


# 5 75

In [8]:
# Calculate the differences and create new columns
df_rC4_5_75['FFC55555_diff'] = df_rC4_5_75['FFC55555'] - df_rC4_5_75['FFC55555_adj']
df_rC4_5_75['Butanol_diff'] = df_rC4_5_75['Butanol'] - df_rC4_5_75['Butanol_adj']
df_rC4_5_75['Decanol_diff'] = df_rC4_5_75['Decanol'] - df_rC4_5_75['Decanol_adj']

In [11]:
df_rC4_5_75['Date'] = pd.to_datetime(df_rC4_5_75['Date'])
df_rC4_5_75.set_index('Date', inplace=True)

In [12]:
# Sum up the hourly data to get daily totals for each component
daily_totals = df_rC4_5_75[['Butanol_diff', 'Decanol_diff']].resample('D').sum()

In [14]:
# Calculate the average daily production
average_daily_totals = daily_totals.mean()

# Count the actual number of days with data
actual_days_with_data = daily_totals.dropna().shape[0]  # dropna() ensures you count only the days with data

# Estimate annual production based on actual data days
estimated_annual_production_butanol = average_daily_totals['Butanol_diff'] * actual_days_with_data
estimated_annual_production_decanol = average_daily_totals['Decanol_diff'] * actual_days_with_data


print(f"Estimated Annual Production for Butanol_diff: {estimated_annual_production_butanol}")
print(f"Estimated Annual Production for Decanol_diff: {estimated_annual_production_decanol}")

Estimated Annual Production for Butanol_diff: -56850.36319843343
Estimated Annual Production for Decanol_diff: -1910.1647709560118
